# Health Check Routines Cookbook

This cookbook demonstrates how to run automated Health Check routines for quantum dot devices using Stanza. Health Check is the critical first step in quantum dot tuning, verifying that gates are functional and determining baseline operating parameters.

## What You'll Learn

- How to configure health check routines in device YAML
- Running the complete health check workflow
- Understanding individual routine results
- Analyzing gate characterization data
- Troubleshooting common issues

## Overview of Health Check Workflow

Health check follows a systematic workflow:

1. **Noise Floor Measurement** - Establish baseline measurement noise
2. **Leakage Test** - Detect electrical shorts between contacts and gates
3. **Global Accumulation** - Determine global turn-on voltage
4. **Reservoir Characterization** - Characterize individual reservoir gates
5. **Finger Gate Characterization** - Characterize plungers and barriers

Each step builds on previous results, establishing a complete picture of device health.

## Setup

First, let's import the necessary modules:

**Note**: Stanza automatically logs all measurements and analysis results when you provide a `DataLogger` as a resource to the `RoutineRunner`. All sweep data, fit parameters, and metadata are saved with timestamps to the specified directory.

In [ ]:
import matplotlib.pyplot as plt

from stanza.routines import RoutineRunner
from stanza.utils import load_device_config

## 1. Device Configuration

Health check routines are configured in your device YAML file. Here's a complete example:

```yaml
name: "My Quantum Device"

gates:
  R1: {type: RESERVOIR, control_channel: 1, measure_channel: 1, v_lower_bound: -3.0, v_upper_bound: 3.0}
  R2: {type: RESERVOIR, control_channel: 2, measure_channel: 2, v_lower_bound: -3.0, v_upper_bound: 3.0}
  B1: {type: BARRIER, control_channel: 3, measure_channel: 3, v_lower_bound: -3.0, v_upper_bound: 3.0}
  P1: {type: PLUNGER, control_channel: 4, measure_channel: 4, v_lower_bound: -3.0, v_upper_bound: 3.0}

contacts:
  SOURCE: {type: SOURCE, control_channel: 5, measure_channel: 5, v_lower_bound: -3.0, v_upper_bound: 3.0}
  DRAIN: {type: DRAIN, control_channel: 6, measure_channel: 6, v_lower_bound: -3.0, v_upper_bound: 3.0}

routines:
  - name: device_health_check
    routines:
      - name: noise_floor_measurement
        parameters:
          measure_electrode: DRAIN
          num_points: 100

      - name: leakage_test
        parameters:
          leakage_threshold_resistance: 50e6  # 50 MOhm
          leakage_threshold_count: 0
          num_points: 10

      - name: global_accumulation
        parameters:
          measure_electrode: DRAIN
          step_size: 0.01  # 10 mV steps
          bias_gate: SOURCE
          bias_voltage: 0.01

      - name: reservoir_characterization
        parameters:
          measure_electrode: DRAIN
          step_size: 0.01
          bias_gate: SOURCE
          bias_voltage: 0.01

      - name: finger_gate_characterization
        parameters:
          measure_electrode: DRAIN
          step_size: 0.01
          bias_gate: SOURCE
          bias_voltage: 0.01

instruments:
  - name: qdac2
    type: GENERAL
    driver: qdac2
    ip_addr: 192.168.1.100
    slew_rate: 1.0
    nplc: 10
```

Save this to `device.yaml` in your working directory.

## 2. Running Complete Health Check

The simplest way to run health check is to execute all routines in sequence:

In [ ]:
# Load device configuration
config = load_device_config("device.yaml")

# Create runner
runner = RoutineRunner(configs=[config])

# Run all health check routines
results = runner.run_all(parent_routine="device_health_check")

print("Health check complete!")
print(f"Available results: {list(results.keys())}")

## 3. Accessing Individual Results

After running health check, you can access results from each routine:

In [ ]:
# Noise floor measurement
noise_floor = results["noise_floor_measurement"]
print(f"Noise floor: {noise_floor['current_std']:.2e} A")
print(f"Mean current: {noise_floor['current_mean']:.2e} A")

# Leakage test
leakage = results["leakage_test"]
print("\nLeakage test passed!")
print(f"Max safe voltage bound: {leakage['max_safe_voltage_bound']:.3f} V")
print(f"Min safe voltage bound: {leakage['min_safe_voltage_bound']:.3f} V")

# Global accumulation
global_turn_on = results["global_accumulation"]
print(f"\nGlobal turn-on voltage: {global_turn_on['global_turn_on_voltage']:.3f} V")

# Reservoir characterization
reservoir_v_cutoffs = results["reservoir_characterization"][
    "reservoir_characterization"
]
print("\nReservoir turn-on voltages:")
for reservoir, v_cutoff in reservoir_v_cutoffs.items():
    print(f"  {reservoir}: {v_cutoff:.3f} V")

# Finger gate characterization
finger_v_cutoffs = results["finger_gate_characterization"][
    "finger_gate_characterization"
]
print("\nFinger gate turn-on voltages:")
for gate, v_cutoff in finger_v_cutoffs.items():
    print(f"  {gate}: {v_cutoff:.3f} V")

## 4. Running Individual Routines

You can also run routines individually for testing or debugging:

In [ ]:
# Run just noise floor measurement
noise_result = runner.run("noise_floor_measurement")
print(f"Noise floor: {noise_result['current_std']:.2e} A")

# Run leakage test (uses noise floor result)
leakage_result = runner.run("leakage_test")
print("Leakage test completed")

# Run global accumulation (uses leakage test results)
global_result = runner.run("global_accumulation")
print(f"Global turn-on: {global_result['global_turn_on_voltage']:.3f} V")

## 5. Visualizing Results

Let's create visualizations of the health check results:

In [ ]:
# Plot turn-on voltages for all gates
fig, ax = plt.subplots(figsize=(10, 6))

# Combine all gate results
all_gates = {}
all_gates.update(reservoir_v_cutoffs)
all_gates.update(finger_v_cutoffs)

gate_names = list(all_gates.keys())
v_cutoffs = list(all_gates.values())

# Plot
colors = [
    "blue" if name.startswith("R") else "green" if name.startswith("B") else "red"
    for name in gate_names
]
ax.bar(gate_names, v_cutoffs, color=colors, alpha=0.7)
ax.axhline(
    global_turn_on["global_turn_on_voltage"],
    color="black",
    linestyle="--",
    label=f"Global Turn-on Voltage: {global_turn_on['global_turn_on_voltage']:.3f} V",
)

ax.set_xlabel("Gate")
ax.set_ylabel("Turn-on Voltage (V)")
ax.set_title("Gate Characterization Results")
ax.legend()
ax.grid(True, alpha=0.3)

# Add legend for colors
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="blue", alpha=0.7, label="Reservoir"),
    Patch(facecolor="green", alpha=0.7, label="Barrier"),
    Patch(facecolor="red", alpha=0.7, label="Plunger"),
]
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.show()

## 6. Automatic Data Logging

When you run health check routines, Stanza automatically logs all measurements and analysis results to disk. The data is organized in the following directory structure:

```
./data/
└── device_health_check/
    ├── noise_floor_measurement/
    │   ├── noise_floor_measurement.h5
    │   ├── session_metadata.json
    │   ├── measurement.jsonl
    │   └── analysis.jsonl
    ├── leakage_test/
    │   ├── leakage_test.h5
    │   ├── session_metadata.json
    │   ├── measurement.jsonl
    │   └── analysis.jsonl
    ├── global_accumulation/
    │   ├── global_accumulation.h5
    │   ├── session_metadata.json
    │   ├── sweep.jsonl
    │   └── analysis.jsonl
    ├── reservoir_characterization/
    │   ├── reservoir_characterization.h5
    │   ├── session_metadata.json
    │   ├── sweep.jsonl
    │   └── analysis.jsonl
    └── finger_gate_characterization/
        ├── finger_gate_characterization.h5
        ├── session_metadata.json
        ├── sweep.jsonl
        └── analysis.jsonl
```

Each routine session contains:
- **HDF5 file (.h5)**: Binary data for efficient storage and access
- **session_metadata.json**: Timestamps, device config, routine parameters
- **measurement/sweep.jsonl**: Raw measurement data (voltages, currents, leakage matrices)
- **analysis.jsonl**: Fit parameters, turn-on voltages, statistics

## 7. Error Handling

Health check routines can fail for various reasons. Here's how to handle errors:

In [ ]:
from stanza.exceptions import RoutineError

try:
    # Run health check
    results = runner.run_all(parent_routine="device_health_check")
    print("Health check passed!")

except RoutineError as e:
    print(f"Configuration error: {e}")
    print("Check your device.yaml file for correct parameters.")

except ValueError as e:
    print(f"Curve fit failed: {e}")
    print("Possible solutions:")
    print("  - Check device connectivity")
    print("  - Verify measure_electrode is correct")
    print("  - Increase voltage range")
    print("  - Reduce step_size for better resolution")

except Exception as e:
    print(f"Unexpected error: {e}")
    print("Check instrument connections and configuration.")

## 8. Parameter Tuning

Different devices may require different parameters. Here's how to adjust them:

### Adjusting Step Size

Trade-off between resolution and measurement time:

```yaml
# Fast (coarse) sweep - 50 mV steps
- name: global_accumulation
  parameters:
    step_size: 0.05

# Standard sweep - 10 mV steps
- name: global_accumulation
  parameters:
    step_size: 0.01

# High resolution sweep - 1 mV steps
- name: global_accumulation
  parameters:
    step_size: 0.001
```

### Adjusting Leakage Threshold

For devices with known crosstalk:

```yaml
# Strict threshold - fail on any leakage
- name: leakage_test
  parameters:
    leakage_threshold_resistance: 100e6  # 100 MOhm
    leakage_threshold_count: 0

# Relaxed threshold - allow some crosstalk
- name: leakage_test
  parameters:
    leakage_threshold_resistance: 10e6  # 10 MOhm
    leakage_threshold_count: 2  # Allow 2 leaky pairs
```

## 9. Advanced - Understanding Pinch-off Analysis

Health check routines use pinch-off curve fitting to extract turn-on voltages. The model is:

$$I(V) = a \cdot (1 + \tanh(b \cdot V + c))$$

Where:
- **a**: Amplitude parameter
- **b**: Slope parameter  
- **c**: Offset parameter

The routine extracts:
- **v_cut_off (cut-off)**: Where device turns off (minimum of second derivative)
- **v_transition (transition)**: Midpoint of transition from cut-off to saturation (maximum of first derivative)
- **v_saturation (saturation)**: Where device is fully on (maximum of second derivative)

## Troubleshooting Guide

### Common Issues

**1. Curve fit failures**
- Check device connectivity and grounding
- Verify measure_electrode matches physical setup
- Increase voltage range (v_lower_bound, v_upper_bound)
- Reduce step_size for better resolution

**2. Leakage test failures**
- Check for physical shorts on device
- Increase leakage_threshold_resistance for known crosstalk
- Set leakage_threshold_count > 0 to allow some leakage
- Verify cable connections and shielding

**3. Missing results errors**
- Ensure routines run in correct order (use nested structure)
- Check that prerequisite routines completed successfully
- Use `runner.run_all()` instead of individual `run()` calls

## Best Practices

1. **Always run noise floor first** - Provides current threshold for leakage testing
2. **Check leakage before characterization** - Running on leaky device can damage gates
3. **Use appropriate step sizes** - Balance resolution vs measurement time
4. **Version control device YAML** - Keep configuration alongside data
5. **Monitor fit quality** - Check for curve fit warnings
6. **Use nested routine structure** - Ensures correct ordering
7. **Review logged data** - All measurements automatically saved to ./data/ directory

## Next Steps

After successful health check, you can:
- Use turn-on voltages as starting points for tuning
- Run charge stability diagram measurements
- Begin quantum dot formation routines
- Perform more detailed gate characterization

See other cookbooks for:
- Charge stability diagrams
- Quantum dot tuning
- Custom routine development

## References

- [Stanza Health Check Documentation](https://docs.conductorquantum.com/stanza/latest/core-concepts/routines/health-check)
- [BATIS Framework](https://arxiv.org/abs/2412.07676)
- [Autonomous Quantum Dot Tuning](https://doi.org/10.1103/PhysRevApplied.13.054005)